In [ ]:
pip install pyvista

In [ ]:
GNN-PINN resolution Elastic 2D Dirichlet-Neumann problem

In [ ]:
import numpy as np
import tensorflow as tf
import networkx as nx
import pyvista as pv
import matplotlib.pyplot as plt

# Generate lattice graph with defined limits
def generate_lattice_graph(H, L, xlim, ylim):
    G = nx.grid_2d_graph(L, H)
    x_range = np.linspace(xlim[0], xlim[1], L)
    y_range = np.linspace(ylim[0], ylim[1], H)
    pos = {node: np.array([x_range[node[0]], y_range[node[1]]]) for node in G.nodes()}
    edges = list(G.edges())
    return G, pos, edges

# Create dataset
def create_dataset(G, pos):
    nodes = list(G.nodes())
    edges = list(G.edges())
    X = np.array([pos[node] for node in nodes], dtype=np.float32)
    edge_indices = np.array([[nodes.index(e[0]), nodes.index(e[1])] for e in edges])
    return X, edge_indices

# Problem dimensions and domain
H, L = 50, 100
xlim, ylim = (0, 1), (0, 0.2)

# Forces and material parameters
force_x, force_y = 0.0, -0.05
E, nu = 1.0, 0.3

# Generate graph and data
G, pos, edges = generate_lattice_graph(H, L, xlim, ylim)
X, edge_indices = create_dataset(G, pos)

# Graph Neural Network
class GraphNeuralNetwork(tf.keras.Model):
    def __init__(self, output_dim):
        super(GraphNeuralNetwork, self).__init__()
        self.dense1 = tf.keras.layers.Dense(16, activation='tanh')
        self.dense2 = tf.keras.layers.Dense(16, activation='tanh')
        self.dense3 = tf.keras.layers.Dense(output_dim)

    def call(self, X):
        X = self.dense1(X)
        X = self.dense2(X)
        X = self.dense3(X)
        return X

# Elasticity PDE loss
def loss_fn(model, X, f_x, f_y, E, nu):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(X)
        u = model(X)
        u_x, u_y = u[:, 0:1], u[:, 1:2]

        u_x_x = tape.gradient(u_x, X)[:, 0:1]
        u_x_y = tape.gradient(u_x, X)[:, 1:2]
        u_y_x = tape.gradient(u_y, X)[:, 0:1]
        u_y_y = tape.gradient(u_y, X)[:, 1:2]

        eps_xx = u_x_x
        eps_yy = u_y_y
        eps_xy = 0.5 * (u_x_y + u_y_x)

        sigma_xx = (E / (1 - nu**2)) * (eps_xx + nu * eps_yy)
        sigma_yy = (E / (1 - nu**2)) * (eps_yy + nu * eps_xx)
        sigma_xy = (E / (1 + nu)) * eps_xy

    with tf.GradientTape() as tape_sigma_x:
        tape_sigma_x.watch(X)
        div_sigma_x = tape.gradient(sigma_xx, X)[:, 0:1] + tape.gradient(sigma_xy, X)[:, 1:2]

    with tf.GradientTape() as tape_sigma_y:
        tape_sigma_y.watch(X)
        div_sigma_y = tape.gradient(sigma_yy, X)[:, 1:2] + tape.gradient(sigma_xy, X)[:, 0:1]

    loss_equilibrium_x = div_sigma_x + f_x
    loss_equilibrium_y = div_sigma_y + f_y

    pde_loss = tf.reduce_mean(tf.square(loss_equilibrium_x)) + tf.reduce_mean(tf.square(loss_equilibrium_y))
    return pde_loss

# Boundary loss for Neumann condition (traction)
def boundary_loss(model, X):
    right_boundary = tf.where(tf.math.abs(X[:, 0] - tf.reduce_max(X[:, 0])) < 1e-6)
    X_b = tf.gather(X, right_boundary[:, 0])

    with tf.GradientTape(persistent=True) as tape:
        tape.watch(X)
        u = model(X)
        u_x, u_y = u[:, 0:1], u[:, 1:2]

        u_x_x = tape.gradient(u_x, X)[:, 0:1]
        u_x_y = tape.gradient(u_x, X)[:, 1:2]
        u_y_x = tape.gradient(u_y, X)[:, 0:1]
        u_y_y = tape.gradient(u_y, X)[:, 1:2]

        eps_xx = u_x_x
        eps_yy = u_y_y
        eps_xy = 0.5 * (u_x_y + u_y_x)

        sigma_xx = (E / (1 - nu**2)) * (eps_xx + nu * eps_yy)
        sigma_yy = (E / (1 - nu**2)) * (eps_yy + nu * eps_xx)
        sigma_xy = (E / (1 + nu)) * eps_xy

    sigma_xx_b = tf.gather(sigma_xx, right_boundary[:, 0])
    sigma_xy_b = tf.gather(sigma_xy, right_boundary[:, 0])

    # Normal vector on right boundary: n = [1, 0]
    t_x = sigma_xx_b
    t_y = sigma_xy_b

    t_x_target = tf.zeros_like(t_x)
    t_y_target = tf.ones_like(t_y) * force_y

    loss = tf.reduce_mean(tf.square(t_x - t_x_target)) + tf.reduce_mean(tf.square(t_y - t_y_target))
    return loss

# Dirichlet condition (optional left-fixed support)
def dirichlet_loss(model, X):
    left_boundary = tf.where(tf.math.abs(X[:, 0] - tf.reduce_min(X[:, 0])) < 1e-6)
    u_left = tf.gather(model(X), left_boundary[:, 0])
    return tf.reduce_mean(tf.square(u_left))

# Training loop
def train(model, X, epochs=2000, lr=0.001):
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    history = []

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss_pde = loss_fn(model, X, force_x, force_y, E, nu)
            loss_neumann = boundary_loss(model, X)
            loss_dirichlet = dirichlet_loss(model, X)
            loss = loss_pde + loss_neumann + loss_dirichlet

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        history.append(loss.numpy())

        if epoch % 200 == 0:
            print(f"Epoch {epoch}, Total Loss: {loss.numpy():.5f}, PDE: {loss_pde.numpy():.5f}, Neumann: {loss_neumann.numpy():.5f}, Dirichlet: {loss_dirichlet.numpy():.5f}")

    return history

# Initialize and train model
output_dim = 2
model = GraphNeuralNetwork(output_dim)
loss_history = train(model, X)

# Plot training loss
plt.figure()
plt.plot(loss_history)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.show()

# Predict displacement
u_pred = model(tf.convert_to_tensor(X, dtype=tf.float32)).numpy()

# Export to VTK
def export_to_vtk(X, u_pred, L, H, filename="displacement.vtk"):
    grid_x = L
    grid_y = H
    x, y = X[:, 0], X[:, 1]
    u_x = u_pred[:, 0]
    u_y = u_pred[:, 1]

    x_grid = x.reshape((grid_x, grid_y)).T
    y_grid = y.reshape((grid_x, grid_y)).T
    u_x_grid = u_x.reshape((grid_x, grid_y)).T
    u_y_grid = u_y.reshape((grid_x, grid_y)).T

    grid = pv.StructuredGrid(x_grid, y_grid, np.zeros_like(x_grid))
    grid["Displacement_X"] = u_x_grid.ravel()
    grid["Displacement_Y"] = u_y_grid.ravel()
    grid.save(filename)
    print(f"VTK file saved to {filename}")

export_to_vtk(X, u_pred, L, H)

# Plot displacement field
plt.figure(figsize=(6, 2))
plt.quiver(X[:, 0], X[:, 1], u_pred[:, 0], u_pred[:, 1], angles='xy', scale_units='xy', scale=1, color='b')
plt.title("Displacement Field")
plt.xlabel("X")
plt.ylabel("Y")
plt.axis("equal")
plt.show()


GNN-PINN resolution Elastic 2D Dirichlet-Dirichlet problem

In [ ]:
import numpy as np
import tensorflow as tf
import networkx as nx
import pyvista as pv
import matplotlib.pyplot as plt

# Generate lattice graph
def generate_lattice_graph(H, L, xlim, ylim):
    G = nx.grid_2d_graph(L, H)
    x_range = np.linspace(xlim[0], xlim[1], L)
    y_range = np.linspace(ylim[0], ylim[1], H)
    pos = {node: np.array([x_range[node[0]], y_range[node[1]]]) for node in G.nodes()}
    edges = list(G.edges())
    return G, pos, edges

# Create dataset
def create_dataset(G, pos):
    nodes = list(G.nodes())
    edges = list(G.edges())
    X = np.array([pos[node] for node in nodes], dtype=np.float32)
    edge_indices = np.array([[nodes.index(e[0]), nodes.index(e[1])] for e in edges])
    return X, edge_indices

# Problem setup
H, L = 50, 100
xlim, ylim = (0, 1), (0, 0.2)
force_x, force_y = 0.0, 0.05
E, nu = 1.0, 0.3

# Generate data
G, pos, edges = generate_lattice_graph(H, L, xlim, ylim)
X, edge_indices = create_dataset(G, pos)

# GNN model
class GraphNeuralNetwork(tf.keras.Model):
    def __init__(self, output_dim):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(16, activation='tanh')
        self.dense2 = tf.keras.layers.Dense(16, activation='tanh')
        self.dense3 = tf.keras.layers.Dense(output_dim)

    def call(self, X):
        X = self.dense1(X)
        X = self.dense2(X)
        X = self.dense3(X)
        return X

# PDE loss
def loss_fn(model, X, f_x, f_y, E, nu):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(X)
        u = model(X)
        u_x, u_y = u[:, 0:1], u[:, 1:2]
        u_x_x = tape.gradient(u_x, X)[:, 0:1]
        u_x_y = tape.gradient(u_x, X)[:, 1:2]
        u_y_x = tape.gradient(u_y, X)[:, 0:1]
        u_y_y = tape.gradient(u_y, X)[:, 1:2]

        eps_xx = u_x_x
        eps_yy = u_y_y
        eps_xy = 0.5 * (u_x_y + u_y_x)

        sigma_xx = (E / (1 - nu**2)) * (eps_xx + nu * eps_yy)
        sigma_yy = (E / (1 - nu**2)) * (eps_yy + nu * eps_xx)
        sigma_xy = (E / (1 + nu)) * eps_xy

    with tf.GradientTape() as tape_sigma_x:
        tape_sigma_x.watch(X)
        div_sigma_x = tape.gradient(sigma_xx, X)[:, 0:1] + tape.gradient(sigma_xy, X)[:, 1:2]

    with tf.GradientTape() as tape_sigma_y:
        tape_sigma_y.watch(X)
        div_sigma_y = tape.gradient(sigma_yy, X)[:, 1:2] + tape.gradient(sigma_xy, X)[:, 0:1]

    loss_equilibrium_x = div_sigma_x + f_x
    loss_equilibrium_y = div_sigma_y + f_y

    return tf.reduce_mean(tf.square(loss_equilibrium_x)) + tf.reduce_mean(tf.square(loss_equilibrium_y))

# Dirichlet loss for both left and right boundaries
def dirichlet_loss(model, X):
    xmin, xmax = tf.reduce_min(X[:, 0]), tf.reduce_max(X[:, 0])
    left_b = tf.where(tf.abs(X[:, 0] - xmin) < 1e-6)[:, 0]
    right_b = tf.where(tf.abs(X[:, 0] - xmax) < 1e-6)[:, 0]
    u_left = tf.gather(model(X), left_b)
    u_right = tf.gather(model(X), right_b)
    return tf.reduce_mean(tf.square(u_left)) + tf.reduce_mean(tf.square(u_right))

# Training loop
def train(model, X, epochs=2000, lr=0.001):
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    history = []

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss_pde = loss_fn(model, X, force_x, force_y, E, nu)
            loss_bc = dirichlet_loss(model, X)
            total_loss = loss_pde + loss_bc

        grads = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        history.append(total_loss.numpy())

        if epoch % 200 == 0:
            print(f"Epoch {epoch}: Total Loss = {total_loss.numpy():.6f}, PDE = {loss_pde.numpy():.6f}, Dirichlet = {loss_bc.numpy():.6f}")

    return history

# Initialize and train
model = GraphNeuralNetwork(output_dim=2)
loss_history = train(model, X)

# Plot training loss
plt.figure()
plt.plot(loss_history)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.show()

# Predict displacement
u_pred = model(tf.convert_to_tensor(X, dtype=tf.float32)).numpy()

# Export to VTK
def export_to_vtk(X, u_pred, L, H, filename="displacement.vtk"):
    x, y = X[:, 0], X[:, 1]
    u_x, u_y = u_pred[:, 0], u_pred[:, 1]
    x_grid = x.reshape((L, H)).T
    y_grid = y.reshape((L, H)).T
    u_x_grid = u_x.reshape((L, H)).T
    u_y_grid = u_y.reshape((L, H)).T

    grid = pv.StructuredGrid(x_grid, y_grid, np.zeros_like(x_grid))
    grid["Displacement_X"] = u_x_grid.ravel()
    grid["Displacement_Y"] = u_y_grid.ravel()
    grid.save(filename)
    print(f"VTK file saved to {filename}")

export_to_vtk(X, u_pred, L, H)

# Plot displacement field
plt.figure(figsize=(6, 2))
plt.quiver(X[:, 0], X[:, 1], u_pred[:, 0], u_pred[:, 1], angles='xy', scale_units='xy', scale=1, color='b')
plt.title("Displacement Field")
plt.xlabel("X")
plt.ylabel("Y")
plt.axis("equal")
plt.show()
